In [2]:
# Importing libraries

import os
import base64
import numpy as np
import re
import random
import json
import csv

import torch
import logging

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [64]:
def get_data(file,error):
    
    list_values = []
    list_normal = []
    
    with open(file) as f:
        reader = csv.reader(f, delimiter='\n')
        for line in reader:
            new_list = line[0].split('"')
            try:
                list_values.extend(new_list[error].split(" "))
                list_normal.append(new_list[error])
            except IndexError:
                pass
        f.close()
    
    return list_values, list_normal
    
data_all, sent_all = get_data("../datasets_machine_translation/train_truth_performed.csv",0)
data_errors, sent_errors = get_data("../datasets_machine_translation/train_truth_performed_errors_only.csv",1)
#data_errors = get_data("../datasets_machine_translation/train_truth_performed.csv",1)

In [59]:
data_errors

['ти',
 'с',
 'розпитав',
 'як',
 'дотуду',
 'їхати',
 'у',
 'поході',
 'брали',
 'участь',
 'князі',
 'андрій',
 'і',
 'олександер',
 'так',
 'званий',
 'невський',
 'а',
 'петроспогрецькому',
 'є',
 'камінь',
 'а',
 'вітця',
 'твого',
 'хоть',
 'ближчіймахнув',
 'до',
 'нього',
 'маленький',
 'старчик',
 'у',
 'довгій',
 'непідперезаній',
 'сорочці',
 'лізь',
 'речу',
 'заквапив',
 'товариша',
 'людота',
 'бо',
 'за',
 'півгоні',
 'не',
 'з',
 'у',
 'чулося',
 'стримане',
 'крякання',
 'та',
 'се',
 'замикалась',
 'одна',
 'з',
 'поміж',
 'служниць',
 'і',
 'поквапилася',
 'прибрати',
 'таріль',
 'хотів',
 'бим',
 'ректи',
 'та',
 'не',
 'відаю',
 'як',
 'би',
 'сте',
 'подивилися',
 'на',
 'то',
 'те',
 'й',
 'нарешті',
 'таки',
 'зловчився',
 'поставити',
 'ногу',
 'на',
 'стременота',
 'людота',
 'вхопив',
 'коня',
 'за',
 'вузду',
 'годой',
 'сидів',
 'як',
 'не',
 'живий',
 'поганці',
 'знущалися',
 'з',
 'його',
 'віри',
 'коли',
 'я',
 'повідаю',
 'тобі',
 'щось',
 'ти',
 'не'

In [60]:
def embeddings(data_all,data_e):
    """Gets embeddings from dataset.
    
    Parameters
    ----------
        
    Returns
    -------
    embeddings : list
        List of embeddings.
    """

    # Single list of sentences
    sentences_all = list(set(data_all))
    sentences_e = list(set(data_e))
    
    print("Getting sentences done")

    #Compute embeddings
    embeddings_all = model.encode(sentences_all, convert_to_tensor=True)
    embeddings_e = model.encode(sentences_e, convert_to_tensor=True)
    
    print("Computing embeddings done")
    
    return embeddings_all, embeddings_e

emb_all, emb_e = embeddings(data_all,data_errors)

Getting sentences done
Computing embeddings done


In [61]:
def cosine(embeddings_all,embeddings_e):
    
    #Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(embeddings_e, embeddings_all)

    print("Computing cosine scores done")
    
    return cosine_scores

cos = cosine(emb_all,emb_e)
#%store cos

Computing cosine scores done


In [62]:
def pairs(cosine_scores):
    
    #Find the pairs with the highest cosine similarity scores
    pairs = []
    for i in range(len(cosine_scores)-1):
        print(i,"out of",len(cosine_scores), end='\r')
        
        for j in range(i+1, len(cosine_scores)):
            pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

    print("Finding pairs done")
    
    return pairs

#%store -r cos
pa = pairs(cos)

Finding pairs done12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854 out of 12854


In [54]:
def sort_pairs(pairs):

    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
        
    print("Sorting pairs done")

    return pairs

#pai = sort_pairs(pa)

In [ ]:
def print_pairs(data,pairs):
    
    #sentences = list(set(data_np))+list(set(data_frcnn))
    sentences = list(set(data))
    
    print("Getting sentences done")
    
    for pair in pairs:
        i, j = pair['index']
        if pair["score"] > 0.9:
            print("{} \t{} \tScore: {:.4f}".format(sentences[i], sentences[j], pair['score']))
    
print_pairs(data_all,pa)

Getting sentences done
 	петлицями 	Score: 1.0000
 	птиця 	Score: 0.9167
 	полицями 	Score: 0.9394
 	пальцями 	Score: 0.9241
строчити 	строчить 	Score: 0.9416
строчити 	старчик 	Score: 0.9113
строчити 	сестрички 	Score: 0.9082
строчити 	старости 	Score: 0.9077
петлицями 	лупили 	Score: 0.9421
петлицями 	лапами 	Score: 0.9300
петлицями 	листопада 	Score: 0.9238
петлицями 	ляпаса 	Score: 0.9050
петлицями 	лопатку 	Score: 0.9246
петлицями 	лопату 	Score: 0.9462
петлицями 	лопата 	Score: 0.9512
петлицями 	лапи 	Score: 0.9546
довірливо 	новому 	Score: 1.0000
довірливо 	нову 	Score: 0.9044
крутнув 	прикинувся 	Score: 1.0000
крутнув 	припинився 	Score: 0.9050
крутнув 	прокинулися 	Score: 0.9246
крутнув 	прокинутися 	Score: 0.9349
крутнув 	принялися 	Score: 0.9012
крутнув 	прокинувся 	Score: 0.9739
новому 	троянцям 	Score: 1.0000
новому 	теремця 	Score: 0.9070
прикинувся 	петренком 	Score: 1.0000
прикинувся 	петренко 	Score: 0.9340
прикинувся 	померк 	Score: 0.9139
прикинувся 	помрете 	Score: 

лупили 	царського 	Score: 0.9258
кавалер 	помітив 	Score: 0.9421
кавалер 	поцілувати 	Score: 0.9215
кавалер 	помістили 	Score: 0.9414
станеш 	переїжджала 	Score: 1.0000
станеш 	пережила 	Score: 0.9101
станеш 	приїжджав 	Score: 0.9275
збігла 	успішно 	Score: 1.0000
збігла 	неспішно 	Score: 0.9022
збігла 	успішної 	Score: 0.9551
збігла 	успішнішим 	Score: 0.9148
кресала 	два 	Score: 1.0000
кресала 	давав 	Score: 0.9547
кресала 	давай 	Score: 0.9167
кресала 	дива 	Score: 0.9501
видобути 	зрадникам 	Score: 0.9562
змішуємося 	масляної 	Score: 1.0000
змішуємося 	маминої 	Score: 0.9181
змішуємося 	милостині 	Score: 0.9088
подробицями 	чіткої 	Score: 0.9092
подробицями 	чїтує 	Score: 0.9006
подробицями 	честі 	Score: 0.9033
проїзні 	відновлення 	Score: 1.0000
звертали 	світів 	Score: 1.0000
звертали 	святої 	Score: 0.9055
звертали 	світової 	Score: 0.9348
звертали 	сумнівів 	Score: 0.9266
звертали 	сумоїстів 	Score: 0.9113
звертали 	сів 	Score: 0.9120
звертали 	сиві 	Score: 0.9149
звертали 	св

варіанти 	прикріплено 	Score: 0.9017
варіанти 	персоналі 	Score: 0.9260
варіанти 	покірно 	Score: 0.9307
варіанти 	пірко 	Score: 0.9127
варіанти 	перевірку 	Score: 0.9016
варіанти 	піроксилін 	Score: 0.9017
варіанти 	прикрості 	Score: 0.9058
сим 	рухами 	Score: 1.0000
сим 	рухома 	Score: 0.9503
сим 	рухом 	Score: 0.9124
сраці 	зброя 	Score: 0.9227
мяз 	замислитись 	Score: 1.0000
мяз 	замикалась 	Score: 0.9163
мяз 	замислившись 	Score: 0.9165
мяз 	займатись 	Score: 0.9568
вивідцям 	брак 	Score: 1.0000
вивідцям 	бак 	Score: 0.9444
вивідцям 	бурлаки 	Score: 0.9029
вивідцям 	бароко 	Score: 0.9037
вивідцям 	бик 	Score: 0.9067
вивідцям 	бурки 	Score: 0.9259
вивідцям 	бабак 	Score: 0.9094
козацької 	дамо 	Score: 1.0000
козацької 	даймьо 	Score: 0.9151
козацької 	двома 	Score: 0.9276
козацької 	даром 	Score: 0.9260
козацької 	дума 	Score: 0.9013
козацької 	дома 	Score: 0.9786
козацької 	дном 	Score: 0.9020
мiстечко 	залишила 	Score: 0.9746
мiстечко 	залишився 	Score: 0.9035
мiстечко 	заломивши

закінчити 	заснуємо 	Score: 0.9208
закінчити 	застаємо 	Score: 0.9467
меркурій 	донець 	Score: 1.0000
котрої 	маю 	Score: 1.0000
котрої 	мою 	Score: 0.9177
уривався 	входив 	Score: 1.0000
уривався 	входимо 	Score: 0.9260
уривався 	виходу 	Score: 0.9202
уривався 	вдвох 	Score: 0.9342
уривався 	виходом 	Score: 0.9097
уривався 	виходити 	Score: 0.9051
уривався 	виходи 	Score: 0.9476
уривався 	виходив 	Score: 0.9812
половини 	московщини 	Score: 0.9803
показувалася 	незамінний 	Score: 1.0000
згорів 	похмурим 	Score: 0.9392
згорів 	порухом 	Score: 0.9266
згорів 	порохом 	Score: 0.9247
згорів 	приходу 	Score: 0.9377
згорів 	приходом 	Score: 0.9690
згорів 	похмура 	Score: 0.9147
згорів 	проходимо 	Score: 0.9673
згорів 	переходом 	Score: 0.9294
чхнув 	віків 	Score: 0.9308
чхнув 	виків 	Score: 0.9242
львиця 	головною 	Score: 1.0000
львиця 	готелю 	Score: 0.9049
львиця 	голкою 	Score: 0.9030
львиця 	головою 	Score: 0.9830
махновських 	вухом 	Score: 0.9226
зробилас 	перечитати 	Score: 0.9916
зроби

людиною 	ковалику 	Score: 0.9179
людиною 	куликовську 	Score: 0.9110
тогідний 	формальний 	Score: 1.0000
тогідний 	фатальний 	Score: 0.9192
тогідний 	файний 	Score: 0.9212
розкішний 	таємниця 	Score: 1.0000
розкішний 	таємницю 	Score: 0.9405
венеди 	єдиній 	Score: 1.0000
венеди 	єдиним 	Score: 0.9035
венеди 	єдиної 	Score: 0.9590
венеди 	єдині 	Score: 0.9781
болбочана 	рибу 	Score: 1.0000
болбочана 	риб 	Score: 0.9049
болбочана 	рубай 	Score: 0.9636
болбочана 	роби 	Score: 0.9100
болбочана 	риба 	Score: 0.9182
будинків 	розділі 	Score: 0.9557
будинків 	розїздів 	Score: 1.0000
будинків 	розвідки 	Score: 0.9102
будинків 	різдво 	Score: 0.9247
будинків 	розїзди 	Score: 0.9155
будинків 	різдвом 	Score: 0.9192
будинків 	розмові 	Score: 0.9034
будинків 	розвідку 	Score: 0.9105
будинків 	радзівіл 	Score: 0.9249
будинків 	розстрілів 	Score: 0.9075
будинків 	розмірі 	Score: 0.9089
будинків 	розїзду 	Score: 0.9217
будинків 	розслідуванні 	Score: 0.9161
будинків 	розкидані 	Score: 0.9042
будинків

виглянув 	співвітчизників 	Score: 0.9087
виглянув 	спеції 	Score: 0.9183
виглянув 	січні 	Score: 0.9043
виглянув 	свічі 	Score: 0.9114
неохоче 	дасть 	Score: 1.0000
неохоче 	достать 	Score: 0.9332
неохоче 	дасться 	Score: 0.9198
витичева 	неволі 	Score: 1.0000
витичева 	невдалої 	Score: 0.9318
витичева 	невеликі 	Score: 0.9171
витичева 	невеличкої 	Score: 0.9092
глосарій 	синові 	Score: 0.9071
глосарій 	словників 	Score: 0.9292
глосарій 	сотникові 	Score: 0.9324
глосарій 	совєтській 	Score: 0.9065
шістнадцять 	значними 	Score: 0.9016
шістнадцять 	значить 	Score: 0.9065
шістнадцять 	знаючи 	Score: 0.9359
магматичний 	вживанням 	Score: 0.9661
магматичний 	вживання 	Score: 1.0000
виє 	всякії 	Score: 0.9274
виє 	всячині 	Score: 0.9009
виє 	виків 	Score: 0.9169
виє 	високі 	Score: 0.9462
виє 	великі 	Score: 0.9030
виє 	військ 	Score: 0.9128
іваном 	полигався 	Score: 1.0000
іваном 	палилося 	Score: 0.9033
іваном 	посилювався 	Score: 0.9062
іваном 	похвалився 	Score: 0.9057
іваном 	поставилас

парфуми 	знайомі 	Score: 0.9039
парфуми 	значної 	Score: 0.9135
парфуми 	знаменитості 	Score: 0.9005
віплочинку 	здобувши 	Score: 1.0000
віплочинку 	згубивши 	Score: 0.9145
віплочинку 	задушив 	Score: 0.9040
віплочинку 	зробивши 	Score: 0.9095


In [ ]:
def cosine_to_csv(title,data,sent_data):
    
    sentences = list(set(sent_data))
    print("Getting sentences done")
            
    with open(title, 'w') as f:
        writer = csv.writer(f, delimiter=' ')
        #writer.writerows(data)
        writer.writerow(["SentenceA","SentenceB","Score"])
        
        for pair in data:
            i, j = pair["index"]
            if pair["score"] > 0.9:
                writer.writerow([sentences[i],sentences[j], pair['score']])
        
        f.close()
    
cosine_to_csv("cosine_to_csv.csv",pa,data_all)

In [ ]:
def check_errors(data_e,sent_e,pairs):
    for sent in sent_e:
        sent = sent.split(' ')
        for p in pairs:
            if sent in p[1]:
                
check_errors(data_errors,sent_errors,pa)